# Formes and sizes of greenhouses

In this document we explore forms and sizes of greenhouses. Visual exploration of greenhouses are described in [separate doc](https://176.9.38.120/cruncher/notebooks/cannabis/Visual-Interpretation.ipynb).

As we have found in the previose notebook, a lot of greenhouses are rectangular, so we begin the exploration with the next assumption:
  * greengouses are rectangular.

Suppose for simplicity that all greenhouses are rectangular.

We have reprojected the shapefile in UTM-10 projection and calculated perimeter and area of the greenhouses.

In [1]:
gh = read.table('DATA/greenhouses32610.csv', header=T, sep=',')
gh = data.frame(id=gh$OBJECTID, name=gh$Name, s=gh$Shape_Area, p=gh$Shape_Leng)
summary(gh)

gh[gh$s > 3000, ]

       id              name            s                  p           
 Min.   :   1.0   10703  :   3   Min.   :     0.0   Min.   :  0.1465  
 1st Qu.: 861.2   122    :   3   1st Qu.:    35.8   1st Qu.: 26.8455  
 Median :1721.5   129    :   3   Median :    71.0   Median : 39.8066  
 Mean   :1721.5   5485   :   3   Mean   :   989.3   Mean   : 44.8516  
 3rd Qu.:2581.8   85     :   3   3rd Qu.:   132.9   3rd Qu.: 56.8388  
 Max.   :3442.0   100    :   2   Max.   :935297.6   Max.   :403.9714  
                  (Other):3425                                        

,id,name,s,p
206,206,84,849324.640580217,19.16362401697
1104,924,10042,565615.128765039,1.03329644048
1127,947,5009,3461.86631846739,232.39953864673
1589,1469,5134,5167.49569032703,403.97135019799
2698,2608,10557,669703.225971626,49.03945337027
2922,2862,5715,935297.621498672,34.18512703221


We can see that some poligons have toplologocal errors. For example [poligon Name=10042](http://kolesov.nextgis.com/resource/100/display?base=osm-mapnik&lon=-123.9459&lat=40.0911&angle=0&zoom=28&styles=99) has 565615 sq meters area and 1 meter perimeter. Remove such poligons from the data:

In [2]:
gh$s[gh$s > 10000] = NA
gh$s[gh$s < 5] = NA
gh$p[gh$p < 2] = NA
gh = na.omit(gh)

summary(gh)

       id              name            s                  p          
 Min.   :   1.0   1025   :   2   Min.   :   5.351   Min.   :  9.674  
 1st Qu.: 856.8   1026   :   2   1st Qu.:  38.754   1st Qu.: 28.368  
 Median :1735.5   1036   :   2   Median :  73.722   Median : 40.444  
 Mean   :1726.3   3188   :   2   Mean   : 115.649   Mean   : 46.220  
 3rd Qu.:2597.2   3189   :   2   3rd Qu.: 136.344   3rd Qu.: 57.733  
 Max.   :3442.0   5343   :   2   Max.   :5167.496   Max.   :403.971  
                  (Other):3320                                       

If a greenhouse is rectangular, then we can write the next system of equations:

$$
\begin{array}
  & a_1 a_2 = s \\ 
  & 2(a_1+a_2) = p 
\end{array}
$$
Where `a1` is the smallest side of the rectangule, `a2` is the bigest, `s` is the area and `p` is the perimeter.

We can solve the equation:

$$
\begin{array}
  & a_1 = \frac{p}{2}-a_2 \\ 
  & a_2 = \frac{p+\sqrt{p^2 - 16 s}}{4}
\end{array}
$$

In [3]:
gh$a2 = (gh$p + sqrt(gh$p^2 - 16 * gh$s))/4

summary(gh)

Warning message:
In sqrt(gh$p^2 - 16 * gh$s): созданы NaN

       id              name            s                  p          
 Min.   :   1.0   1025   :   2   Min.   :   5.351   Min.   :  9.674  
 1st Qu.: 856.8   1026   :   2   1st Qu.:  38.754   1st Qu.: 28.368  
 Median :1735.5   1036   :   2   Median :  73.722   Median : 40.444  
 Mean   :1726.3   3188   :   2   Mean   : 115.649   Mean   : 46.220  
 3rd Qu.:2597.2   3189   :   2   3rd Qu.: 136.344   3rd Qu.: 57.733  
 Max.   :3442.0   5343   :   2   Max.   :5167.496   Max.   :403.971  
                  (Other):3320                                       
       a2         
 Min.   :  3.087  
 1st Qu.:  9.906  
 Median : 15.079  
 Mean   : 17.775  
 3rd Qu.: 22.988  
 Max.   :171.930  
 NA's   :188      

There are polygon with wrong descriminants (below zero). We cheched some of them, they also have wrong toplogy. So we can ommit them.

In [4]:
gh = na.omit(gh)
gh$a1 = (gh$p)/2 - gh$a2
summary(gh)

       id              name            s                  p         
 Min.   :   1.0   1025   :   2   Min.   :   5.351   Min.   : 10.12  
 1st Qu.: 839.8   1026   :   2   1st Qu.:  39.523   1st Qu.: 29.09  
 Median :1739.5   1036   :   2   Median :  74.784   Median : 41.10  
 Mean   :1725.5   3188   :   2   Mean   : 114.580   Mean   : 46.78  
 3rd Qu.:2620.2   3189   :   2   3rd Qu.: 136.885   3rd Qu.: 58.41  
 Max.   :3442.0   5343   :   2   Max.   :5167.496   Max.   :403.97  
                  (Other):3132                                      
       a2                a1         
 Min.   :  3.087   Min.   : 0.9002  
 1st Qu.:  9.906   1st Qu.: 3.5454  
 Median : 15.079   Median : 4.8056  
 Mean   : 17.775   Mean   : 5.6162  
 3rd Qu.: 22.988   3rd Qu.: 7.0096  
 Max.   :171.930   Max.   :36.3811  
                                    

Draw the sides of the greengouses.

In [5]:
png("Img/rect_sizes.png", width=728, height=728, units="px")
  plot(gh$a1, gh$a2, pch = 19, cex=0.01)
dev.off()

png 
  2

<img src="Img/rect_sizes.png">

We can see that the length parameters have variations but most of the greenhoses are not very big. They are usually 4x10 -- 7x25 meters. But there are very big greenhouses for example 30x170.

The question about form and size have to be explored more, but this plot gives us enought information to continue analysis of the textures (see the contlusion in the [previouse doc](https://176.9.38.120/cruncher/notebooks/cannabis/Visual-Interpretation.ipynb)).